In [29]:
from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr
import webrtcvad
from scipy.stats import spearmanr
import os
import seaborn as sns
import sox
import shutil
from operator import itemgetter
from itertools import groupby
import numpy as np
import shutil
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from scipy.stats import ranksums
import seaborn as sns

import statsmodels.api

import os
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from statsmodels.stats.multitest import fdrcorrection


In [2]:
export_table = True
correlation = 'spearman'  # or 'pearson'

In [11]:
# Load data_frame containing features, speaker ID and labels (i.e., "PD", "CN")
new = pd.read_csv('/export/b15/afavaro/Frontiers/submission/Statistical_Analysis/NEUROVOZ/rhythm_data_whispx.csv')
#new = new.dropna()
new = new.drop(columns=['total_name'])

new = new.dropna()
#new['sp'] = [int(elem.split("_")[2].split("-16")[0])  for elem in new['idx'].tolist()]
new = new.sort_values(by='name', ignore_index=True)
new = new.rename(columns={"name":"ID" })
new = new.drop(columns=['Unnamed: 0'])

new.head()[:2]

,ID,label,task,rhythm_std,rhythm_skew,rhythm_kurt
0,6,PD,concatenateread,10.783627,0.121304,-1.255141
1,7,PD,concatenateread,11.104187,0.162573,-1.212628


In [13]:
# Load clinical data
demo = pd.read_excel("/export/b15/afavaro/Frontiers/submission/Correlation_Analysis/SPANISH/Datos.xlsx")
codes = demo['PAC'].tolist()
updrs =  demo['Escala UPDRS'].tolist()
hoen = demo['Estadio H-Y'].tolist()

In [16]:
# normalize
feats = new.iloc[:, 3:-2] # select only columns containing feature values
info_subject = new.iloc[:, :3] #select only columns containing task and subject-related information.
df_z_scaled = feats.copy()
  
# apply normalization techniques
for column in df_z_scaled.columns:
    df_z_scaled[column] = (df_z_scaled[column] -
                           df_z_scaled[column].mean())/df_z_scaled[column].std()    

normalized = df_z_scaled
final =pd.concat([normalized, info_subject], axis=1)
final.head()[:1]

,rhythm_std,ID,label,task
0,-0.42839,6,PD,concatenateread


In [17]:
# Group by tasks for the analysis
grouped = final.groupby("task")
read = grouped.get_group("concatenateread")
mon = grouped.get_group("ESPONTANEA")

In [19]:
## Combining feature values and clinical scores into a unique data frame.
spk2lab_ = {sp:lab for sp,lab in zip(codes,updrs)}
#read['sp'] = [int(elem.split("_")[2]) for elem in read['idx'].tolist()]
speak = read['ID'].tolist()
etichettex = []
for nome in speak:
    if nome in spk2lab_.keys():
        lav = spk2lab_[nome]
        etichettex.append(([nome, lav]))
    else:
         etichettex.append(([nome, 'Unknown']))
label_new_ = []
for e in etichettex:
    label_new_.append(e[1])
read['updrs'] = label_new_

spk2lab_ = {sp:lab for sp,lab in zip(codes, hoen)}

etichettex = []
for nome in speak:
    if nome in spk2lab_.keys():
        lav = spk2lab_[nome]
        etichettex.append(([nome, lav]))
    else:
         etichettex.append(([nome, 'Unknown']))
label_new_ = []
for e in etichettex:
    label_new_.append(e[1])
read['hoen'] = label_new_



/tmp/3539467.1.g.q/ipykernel_210137/3356042900.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  read['updrs'] = label_new_
/tmp/3539467.1.g.q/ipykernel_210137/3356042900.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  read['hoen'] = label_new_


In [20]:
spk2lab_ = {sp:lab for sp,lab in zip(codes,updrs)}
#mon['sp'] = [int(elem.split("_")[2].split("-16")[0])  for elem in mon['idx'].tolist()]
speak = mon['ID'].tolist()
etichettex = []
for nome in speak:
    if nome in spk2lab_.keys():
        lav = spk2lab_[nome]
        etichettex.append(([nome, lav]))
    else:
         etichettex.append(([nome, 'Unknown']))
label_new_ = []
for e in etichettex:
    label_new_.append(e[1])
mon['updrs'] = label_new_

spk2lab_ = {sp:lab for sp,lab in zip(codes, hoen)}

etichettex = []
for nome in speak:
    if nome in spk2lab_.keys():
        lav = spk2lab_[nome]
        etichettex.append(([nome, lav]))
    else:
         etichettex.append(([nome, 'Unknown']))
label_new_ = []
for e in etichettex:
    label_new_.append(e[1])
mon['hoen'] = label_new_

/tmp/3539467.1.g.q/ipykernel_210137/1161279131.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mon['updrs'] = label_new_
/tmp/3539467.1.g.q/ipykernel_210137/1161279131.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mon['hoen'] = label_new_


In [48]:
# use only PD subjects for correlation analysis
gr_read = read.groupby("label")
pd_read = gr_read.get_group("PD")

gr_mon = mon.groupby("label")
pd_mon = gr_mon.get_group("PD")
pd_mon.head()[:2]

,rhythm_std,ID,label,task,updrs,hoen
25,-1.215151,35,PD,ESPONTANEA,14.0,2.0
28,-0.757056,37,PD,ESPONTANEA,47.0,5.0


In [47]:
pd_mon.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)

/home/afavaro/new_conda/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [24]:
pd_read.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)

In [50]:
## correlation between features and UPDRS 
def compute_correlation(data_frame, num_cols, clinical_score):
    
    feats_col = data_frame.iloc[:, :num_cols] #select only columns containing features
   # print(feats_col)
    updrs_pd = data_frame[clinical_score].tolist()
    feats = feats_col.columns.values.tolist() 
    file = []
    p_vals = []
    
    #print spearman's correlation values and respective p-value
    
    for fea in feats:
        data = feats_col[fea].tolist()
        corr, _ = spearmanr(data, updrs_pd)
        p_vals.append(_)
        file.append((f'Spearm correlation for feats {fea}: p_value {_} and correlation coeff is {corr}'))
   # print(file)
    # Apply correction 
    res = statsmodels.stats.multitest.fdrcorrection(p_vals, alpha=0.05, method='indep', is_sorted=False)   
    ows=np.where(res[1][:,]<0.05)
    l = list(ows[0])
    values = res[1][l]
    for m in zip(l, values):
        print(m, feats[m[0]])   
        
   

In [51]:
compute_correlation(pd_read, 1, 'updrs')

In [52]:
compute_correlation(pd_read, 1, 'hoen')